### Simulacion usando Simpy (caso PIL)

La empresa de leche Pil tiene compradores al por mayor. los cuales realizan pedidos a la empresa que son a su ves redirigidos a los carros repartidores.

Estos carros repartidores inician su jornada con la lista de pedidos y proceden a ir a la seccion de carga segun orden de llegada para cargar los productos que los compradores por mayor pidieron para ese dia.

In [1]:
#instalamos la libreria de sumulacion.
!pip install simpy

In [2]:
# Importamos las bibliotecas necesarias
import random
import math
import simpy

In [3]:
# definimos algunos datos para la simulacion.
num_despachadores = 1
tiempo_carga_min = 10
tiempo_carga_max = 20
tiempo_entre_llegadas = 15
total_repartidores = 5
 
te  = 0.0 # tiempo de espera total

In [4]:
def cargar(repartidor):
    intervalo = (tiempo_carga_max - tiempo_carga_min) * random.random() #intervalo aleatorio
    tiempo_corte = tiempo_carga_min + intervalo 
    yield env.timeout(tiempo_corte) # deja correr el tiempo n minutos
    print(" Carro del repartidor {} cargado en {:.2f} minutos".format(repartidor,tiempo_corte))

def repartidor (env, nombre, despachador ):
    global te
    hora_llegada = env.now # Guarda el minuto de llegada del cliente
    print ("---> {} llego al area de carga en minuto {:.2f}".format(nombre, hora_llegada))
    with despachador.request() as request: # Espera su turno
        yield request # Obtiene turno
        inicia = env.now # Guarda el minuto cuado comienza a ser atendido
        espera = inicia - hora_llegada # Calcula el tiempo que espero
        te = te + espera # Acumula los tiempos de espera
        print ("**** {} pasa con el despachador en minuto {:.2f} habiendo esperado {:.2f}".format(nombre, inicia, espera))
        yield env.process(cargar(nombre)) # Invoca al proceso cargar los productos al carro
        termina = env.now #Guarda el minuto en que termina el proceso carga de productos 
        print ("<--- {} deja el area de carga en minuto {:.2f}".format(nombre, termina))

def principal (env, despachador):
    llegada = 0
    i = 0
    for i in range(total_repartidores):
        R = random.random()
        llegada = -tiempo_entre_llegadas * math.log(R)
        yield env.timeout(llegada)  # Deja transcurrir un tiempo entre uno y otro
        i += 1
        env.process(repartidor(env, 'Repartidor %d' % i, despachador)) 

In [5]:
print ("------------------- Bienvenido Simulacion carga de carros repartidores ------------------")
random.seed (30) # Cualquier valor
env = simpy.Environment() # Crea el objeto entorno de simulacion
despachadores = simpy.Resource(env, num_despachadores) #Crea los recursos (despachadores)
env.process(principal(env, despachadores)) #Invoca el proceso princial
env.run() #Inicia la simulacion

------------------- Bienvenido Simulacion carga de carros repartidores ------------------
---> Repartidor 1 llego al area de carga en minuto 9.27
**** Repartidor 1 pasa con el despachador en minuto 9.27 habiendo esperado 0.00
 Carro del repartidor Repartidor 1 cargado en 10.30 minutos
<--- Repartidor 1 deja el area de carga en minuto 19.57
---> Repartidor 2 llego al area de carga en minuto 27.88
**** Repartidor 2 pasa con el despachador en minuto 27.88 habiendo esperado 0.00
---> Repartidor 3 llego al area de carga en minuto 34.26
 Carro del repartidor Repartidor 2 cargado en 12.10 minutos
<--- Repartidor 2 deja el area de carga en minuto 39.98
**** Repartidor 3 pasa con el despachador en minuto 39.98 habiendo esperado 5.72
 Carro del repartidor Repartidor 3 cargado en 13.97 minutos
<--- Repartidor 3 deja el area de carga en minuto 53.95
---> Repartidor 4 llego al area de carga en minuto 54.62
**** Repartidor 4 pasa con el despachador en minuto 54.62 habiendo esperado 0.00
---> Reparti

In [6]:
print ("Tiempo de espera promedio = {:.2f}".format(te / total_repartidores))

Tiempo de espera promedio = 3.79
